This notebook contains the code of the most similar atlas (SIM) approach 

In [11]:
# Import libraries
from pathlib import Path
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

from metadata import ImageDataset, patient

In [2]:
notebook_path = Path().cwd()
repo_path = notebook_path.parent
print(f'The current directory is: {notebook_path}')

The current directory is: d:\VS_Projects\MISA_FINAL_PROJECT\notebooks


step1: perfoerm preprocessing
step2: perform registration

In [7]:
#instantiate dataset, specify set name
img_training_data = ImageDataset('Training')
im_val_data = ImageDataset('Validation')

#check length
# print(f'len: {img_training_data.len}')
# #get all ids (useful to iterate over all adataset)
# print(f'all ids: {img_training_data.IDs}')
# #get list of all images paths for training
# print(f'images paths: {img_training_data.im_paths()}')
# #same for the labels
# print(f'images paths: {img_training_data.labels_paths()}')


#select one id of patient to check
id = img_training_data.IDs[0]
id_val = im_val_data.IDs[0]
print(f'selected id: {id}')
#istantiate patient object using the id and the previous ImageDataset object
pat = patient(id, img_training_data)
pat_val = patient(id_val, im_val_data)

# pat.show('labels')
# pat.show('im')

# Get the image and the labels
im = pat.im()
labels = pat.labels()

im_val = pat_val.im()
labels_val = pat_val.labels()

selected id: 01


In [13]:
def image_registration(fixedImage, movingImage):
    """Give two images and the registration of both, and its transformation map will be given

    Args:
        fixed_path (str): fixed (template) image
        moving_path (str): moving image (image that will be transformed)

    Returns:
        sitk image, transformix map: transformed image and the transformation map
    """

    #Start registration settings
    elastixImageFilter = sitk.ElastixImageFilter() #Image filter object
    #Defining settings
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)

    #Run registration
    elastixImageFilter.Execute()

    #Get result image
    resultImage = elastixImageFilter.GetResultImage()

    #Transformation map
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()
    transformParameterMap[0]["ResampleInterpolator"] = ["FinalNearestNeighborInterpolator"]
    transformParameterMap[1]["ResampleInterpolator"] = ["FinalNearestNeighborInterpolator"]    
    transformParameterMap[2]["ResampleInterpolator"] = ["FinalNearestNeighborInterpolator"]
    return resultImage, transformParameterMap

In [14]:
fixed_image = im_val
# for each training image, register it to the validation image
for i in len(img_training_data.IDs):
    id = img_training_data.IDs[i]
    pat = patient(id, img_training_data)
    moving_image = pat.im()
    labels = pat.labels()

    for v in len(im_val_data.IDs):
        id_val = im_val_data.IDs[v]
        pat_val = patient(id_val, im_val_data)
        fixed_image = pat_val.im()
        labels_val = pat_val.labels()

        list_labels = []

        resultImage, transformParameterMap = image_registration(fixed_image, moving_image) #Resgiter images using predefined transform map

        transformixImageFilter = sitk.TransformixImageFilter() #Create object transform matrix
        transformixImageFilter.SetTransformParameterMap(transformParameterMap) #Set with found transform
        transformixImageFilter.SetMovingImage(labels_val) #set labels as moving
        transformixImageFilter.Execute() #Tranform labels

        labels_registered = transformixImageFilter.GetResultImage() #Propagated labels

        with open(f'trans_labels.p', 'wb') as handle:   #Save propagated labels as pickle file
            pickle.dump(labels_registered, handle, pickle.HIGHEST_PROTOCOL)
            
        list_labels.append(labels_registered) #Append all propagated labels on a list

# to save all the propagated labels in one file
with open(f'all_trans_labels.p', 'wb') as handle: 
    pickle.dump(list_labels, handle, pickle.HIGHEST_PROTOCOL)